## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn

## Extracting Data and Removing Correlated Variables

In [2]:
name_list = ['id','avg_len','tot_len','avg_rate','tot_rate','review','rating','completion','min_listened','support_req','lv','target']
data = pd.read_csv('Audiobooks_data.csv',header = None, names = name_list)
data.drop('id',axis=1,inplace = True)

In [3]:
data['units_purchased'] = (data['tot_len']/data['avg_len']).astype(int)

In [4]:
data.drop(['avg_len','avg_rate','min_listened'],axis=1,inplace =True)

In [5]:
from sklearn.utils import shuffle
data = shuffle(data).reset_index().drop('index',axis=1)
data

,tot_len,tot_rate,review,rating,completion,support_req,lv,target,units_purchased
0,2160,5.33,0,8.91,0.00,0,168,0,1
1,1188,5.33,0,8.91,0.00,0,0,0,1
2,2160,8.00,1,9.00,0.16,0,199,0,1
3,4428,16.00,0,8.91,0.00,0,0,1,3
4,1620,10.13,0,8.91,0.00,0,0,1,1
...,...,...,...,...,...,...,...,...,...
14079,2160,5.33,0,8.91,0.00,0,0,0,1
14080,2160,7.99,0,8.91,0.00,0,0,1,1
14081,2160,6.13,0,8.91,0.26,0,192,0,1
14082,1620,5.61,0,8.91,0.52,0,60,0,1


## Dataset Balancing

In [6]:
ones = np.sum(data['target'])
zeros = 0
removal_index = []
for i,r in data.iterrows():
    if r['target'] == 0:
        zeros += 1
        if zeros > ones:
            removal_index.append(i)

data.drop(data.index[removal_index], inplace = True)
data.describe()

,tot_len,tot_rate,review,rating,completion,support_req,lv,target,units_purchased
count,4474.000000,4474.000000,4474.000000,4474.000000,4474.000000,4474.000000,4474.000000,4474.000000,4474.000000
mean,1833.127403,8.509611,0.164953,8.909725,0.075311,0.079124,70.434287,0.500000,1.200268
std,874.874972,6.570098,0.371180,0.714516,0.197712,0.404532,92.289370,0.500056,0.561162
min,216.000000,3.860000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1188.000000,5.330000,0.000000,8.910000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1620.000000,6.735000,0.000000,8.910000,0.000000,0.000000,21.000000,0.500000,1.000000
75%,2160.000000,8.610000,0.000000,8.910000,0.000000,0.000000,127.000000,1.000000,1.000000
max,7020.000000,111.470000,1.000000,10.000000,1.000000,8.000000,374.000000,1.000000,6.000000


## Feature Scaling and splitting into Train-Test-Validate

In [7]:
data = shuffle(data).reset_index().drop('index',axis=1)

In [8]:
from mlxtend.preprocessing import standardize
feature_list = ['tot_len','tot_rate','review','rating','completion','support_req','lv','units_purchased']
targets = data['target']
features = standardize(data, columns = feature_list)
features.describe()

,tot_len,tot_rate,review,rating,completion,support_req,lv,units_purchased
count,4.474000e+03,4.474000e+03,4.474000e+03,4.474000e+03,4.474000e+03,4.474000e+03,4.474000e+03,4.474000e+03
mean,1.661985e-16,-5.021443e-14,2.787717e-16,-5.559769e-13,6.883719e-16,3.876227e-16,-1.040989e-16,2.792928e-16
std,1.000112e+00,1.000112e+00,1.000112e+00,1.000112e+00,1.000112e+00,1.000112e+00,1.000112e+00,1.000112e+00
min,-1.848616e+00,-7.077720e-01,-4.444519e-01,-1.107129e+01,-3.809540e-01,-1.956152e-01,-7.632749e-01,-3.569212e-01
25%,-7.374762e-01,-4.840060e-01,-4.444519e-01,3.848095e-04,-3.809540e-01,-1.956152e-01,-7.632749e-01,-3.569212e-01
50%,-2.436362e-01,-2.701344e-01,-4.444519e-01,3.848095e-04,-3.809540e-01,-1.956152e-01,-5.357043e-01,-3.569212e-01
75%,3.736638e-01,1.528138e-02,-4.444519e-01,3.848095e-04,-3.809540e-01,-1.956152e-01,6.129854e-01,-3.569212e-01
max,5.929364e+00,1.567281e+01,2.249962e+00,1.526064e+00,4.677478e+00,1.958252e+01,3.289649e+00,8.554159e+00


In [9]:
test_val_size = int(0.1* features.shape[0])
train_size = features.shape[0] - 2 * test_val_size


test_x, test_y = features[0 : test_val_size], targets[0 : test_val_size]
val_x, val_y = features[test_val_size : 2*test_val_size], targets[test_val_size : 2*test_val_size]
train_x, train_y = features[2*test_val_size :],targets[2*test_val_size :]

# Building and Testing Model


In [16]:
input_size = 8
output_size = 2
hidden_layer_size = 50

batch_size = 100

max_epochs = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    
    tf.keras.layers.Dense(output_size, activation='softmax') 
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_x, 
          train_y, 
          batch_size=batch_size, 
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(val_x, val_y),
          verbose = 2
          )  


Epoch 1/100
36/36 - 0s - loss: 0.5910 - accuracy: 0.6899 - val_loss: 0.5011 - val_accuracy: 0.7740
Epoch 2/100
36/36 - 0s - loss: 0.4837 - accuracy: 0.7595 - val_loss: 0.4517 - val_accuracy: 0.7785
Epoch 3/100
36/36 - 0s - loss: 0.4341 - accuracy: 0.7818 - val_loss: 0.4159 - val_accuracy: 0.8031
Epoch 4/100
36/36 - 0s - loss: 0.4088 - accuracy: 0.7891 - val_loss: 0.3993 - val_accuracy: 0.8076
Epoch 5/100
36/36 - 0s - loss: 0.3963 - accuracy: 0.7908 - val_loss: 0.3904 - val_accuracy: 0.8121
Epoch 6/100
36/36 - 0s - loss: 0.3876 - accuracy: 0.7986 - val_loss: 0.3824 - val_accuracy: 0.8076
Epoch 7/100
36/36 - 0s - loss: 0.3797 - accuracy: 0.8014 - val_loss: 0.3833 - val_accuracy: 0.8143
Epoch 8/100
36/36 - 0s - loss: 0.3729 - accuracy: 0.8089 - val_loss: 0.3726 - val_accuracy: 0.8121
Epoch 9/100
36/36 - 0s - loss: 0.3685 - accuracy: 0.8025 - val_loss: 0.3680 - val_accuracy: 0.8121
Epoch 10/100
36/36 - 0s - loss: 0.3635 - accuracy: 0.8089 - val_loss: 0.3709 - val_accuracy: 0.8098
Epoch 11/

In [17]:
model.evaluate(test_x, test_y)

14/14 [==============================] - 0s 1ms/step - loss: 0.3769 - accuracy: 0.8031


[0.3768601417541504, 0.8031319975852966]